# Домашнее задание №1

В первом ДЗ мы создаём и обустраиваем git-репозиторий, который будем использовать в дальнейшем, загружаем данные о ц.б. SNP500 и криптовалютах, настраиваем автоматическое отображение графиков котировок и частично проводим предобработку и анализ данных. Для того, чтобы всё сделать правильно, я создал небольшой чек-лист ДЗ1, который Вы видите ниже:

~~1. **Создать git-репозиторий**, где будет храниться исходный код вашего проекта. Если вы используете приватный репозиторий – дайте преподавателям курса доступ к нему, для возможности проверки ДЗ.~~ *(Использовал публичный репозиторий)*

~~2.  **Добавить файл лицензии**, который отражает ваш взгляд на конфиденциальность информации, которую вы подготовите в рамках данного курса.~~ \
\
3. **Создать код на Python**, который загрузит на ваш локальный компьютер данные о:\
    a) котировках ценных бумаг из списка **SnP500**; \
    b) котировки **криптовалют** (BTC, ETH, SOL, XRP). \
\
4. Поскольку вам предстоит много работать с ними в дальнейшем, подготовьте **автоматическое отображение графиков** текущей ситуации. \
\
5.  a) Проверьте нет ли в данных **пропусков или ошибок**; \
    b) Проанализируйте **выбросы**; \
    c) Оцените, на самом ли деле это **выбросы или реальные данные**, с которыми предстоит работать. 

*Примечание: поскольку первые два шага делаются вне этого ноутбука, нумерация разделом начинается с цифры "3"*

## 0: Импорты

In [106]:
##%pip install pandas
##%pip install requests
##%pip install bs4

In [107]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

## 3: Загрузка данных

In [108]:
link = "https://www.tradingview.com/symbols/SPX/?ysclid=m3ytbdruxe2541901"
headers_snp_btc = {"user agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 YaBrowser/24.10.0.0 Safari/537.36"}

html = requests.get(url=link, headers=headers_snp_btc)

In [109]:
soup = BeautifulSoup(html.content, "html.parser")
convert = soup.findAll("span", {"translate": "no", "class": "last-JWoJqCpY js-symbol-last"})

In [ ]:
price = convert[0].text # пока не разобрался, как получить само значение
price

''